In [5]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.util import *
from collections import OrderedDict

In [18]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1}, 
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int(input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], 
                                                             filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], 
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            dout = layer.backward(dout)
            
            grads = {}
            grads['W1'] = self.layers['Conv1'].dW
            grads['b1'] = self.layers['Conv1'].db
            grads['W2'] = self.layers['Affine1'].dW
            grads['b2'] = self.layers['Affine1'].db
            grads['W3'] = self.layers['Affine2'].dW
            grads['b3'] = self.layers['Affine2'].db
            
        return grads
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)        
        acc = 0.0        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

In [7]:
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

In [8]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

In [19]:
max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

train loss:2.2992340090887113
=== epoch:1, train acc:0.113, test acc:0.108 ===
train loss:2.2965568754214787
train loss:2.2920512095495242
train loss:2.287509229578048
train loss:2.2757910265627443
train loss:2.2692498258033695
train loss:2.256664705386427
train loss:2.2411012399890105
train loss:2.2137624624163483
train loss:2.1906990721869253
train loss:2.1641595488327927
train loss:2.1095643961258306
train loss:2.0832158623944523
train loss:2.0070956309328842
train loss:1.9538133433365579
train loss:1.874434414151516
train loss:1.891631752410182
train loss:1.756612210478588
train loss:1.7085442857530506
train loss:1.6553480168926111
train loss:1.533967092348805
train loss:1.4483333732699548
train loss:1.3619069822531853
train loss:1.332803180894597
train loss:1.332892516659383
train loss:1.223019704762805
train loss:1.0749038850462036
train loss:0.9445602092269887
train loss:0.9239491807244959
train loss:0.9270720741025867
train loss:0.9235459234455321
train loss:0.8287617542156259


train loss:0.132586200223617
train loss:0.1851725788238325
train loss:0.14826171975113567
train loss:0.15067169266010808
train loss:0.1689947281547308
train loss:0.16670867950998378
train loss:0.17116612259941166
train loss:0.24773907773970485
train loss:0.1358365156853422
train loss:0.17603100025745172
train loss:0.20034555467944254
train loss:0.11713189588328675
train loss:0.2050823897181693
train loss:0.1576927114854853
train loss:0.17019081174858097
train loss:0.2818471398381544
train loss:0.2154190565362754
train loss:0.3554787933873367
train loss:0.2040751039486276
train loss:0.23115493173192556
train loss:0.13513178258908298
train loss:0.10390865238322723
train loss:0.12679240576812123
train loss:0.22016485980094128
train loss:0.08334269537408792
train loss:0.2739779052405958
train loss:0.13918538615972792
train loss:0.14226452339142892
train loss:0.23905216382440414
train loss:0.18545993445628414
train loss:0.0885240636247385
train loss:0.1843291207112145
train loss:0.176835325

train loss:0.047821006374924535
train loss:0.14660991213487737
train loss:0.14630591115950106
train loss:0.12758990676265977
train loss:0.08503199930845103
train loss:0.18001824366165775
train loss:0.1758428035099467
train loss:0.12279107964411941
train loss:0.11117571252370181
train loss:0.1775843930220766
train loss:0.17451898556569292
train loss:0.1171840116828825
train loss:0.13245775354190895
train loss:0.16012900024551344
train loss:0.20672769564639634
train loss:0.08030843121479041
train loss:0.17963982761902958
train loss:0.06021140693099282
train loss:0.09877565123093865
train loss:0.08251822467358344
train loss:0.0653836619822619
train loss:0.18077716650345674
train loss:0.09080647256155544
train loss:0.14787081697275778
train loss:0.11074068648336612
train loss:0.06539977193026304
train loss:0.11977858245947638
train loss:0.2654147803529062
train loss:0.07940904218045297
train loss:0.12470521584524427
train loss:0.08824508248262938
train loss:0.12638523628613793
train loss:0

train loss:0.0996658171183238
train loss:0.0889394537504455
train loss:0.22590376852354613
train loss:0.06835447088522487
train loss:0.04123454917469536
train loss:0.10164491992587323
train loss:0.060975473537203694
train loss:0.03311681217621906
train loss:0.06765871687932115
train loss:0.054815585716908075
train loss:0.05257490801075362
train loss:0.07422796868584261
train loss:0.11462384465906028
train loss:0.08469214307636974
train loss:0.05967538752097004
train loss:0.06076290617918896
train loss:0.0686435391813417
train loss:0.019334655634190755
train loss:0.06995553526549135
train loss:0.1272658588779375
train loss:0.04972487362340872
train loss:0.10188133567872865
train loss:0.03315852690249781
train loss:0.09156577127530124
train loss:0.05742952330392891
train loss:0.06427426123907448
train loss:0.08872468848067702
train loss:0.06799389210800276
train loss:0.06299630710757465
train loss:0.0364400873018796
train loss:0.02521107103256283
train loss:0.23480775111464536
train loss

train loss:0.03724871051484185
train loss:0.10406507183624267
train loss:0.04117004899546026
train loss:0.015933781331160413
train loss:0.08542100025731099
train loss:0.11612760768852615
train loss:0.030516476274244843
train loss:0.10365138996360734
train loss:0.09128010587518627
train loss:0.04554956103272656
train loss:0.06702216612474544
train loss:0.07849504917458394
train loss:0.05443449415940256
train loss:0.06796397497465995
train loss:0.05300751205708927
train loss:0.08875825793875278
train loss:0.025663085618381643
train loss:0.028245916826974606
train loss:0.07414503228703415
train loss:0.0862361744813631
train loss:0.08793683190485697
train loss:0.03888302634257926
train loss:0.0613644334996224
train loss:0.02677522227563797
train loss:0.04763049053133637
train loss:0.04989233340653757
train loss:0.02912927993820456
train loss:0.06235002375380942
train loss:0.08640585143326071
train loss:0.06466468773600588
train loss:0.02027561909492505
train loss:0.1052150864535122
train l

...

train loss:0.0015326649582394132
train loss:0.003317014975233397
train loss:0.006221434614741952
train loss:0.0111015283418447
train loss:0.0015265839376066165
train loss:0.003996411046695229
train loss:0.004513128398763332
train loss:0.0001299360084607832
train loss:0.0006150935218781174
train loss:0.00182020238143949
train loss:0.01033894936475199
train loss:0.0012697866174083152
train loss:0.0005639104581859756
train loss:0.0025186650278215634
train loss:0.0004645715616306796
train loss:0.0009773421027123974
train loss:0.0010847809042049943
train loss:0.006544378120244202
train loss:0.0022248463383270573
train loss:0.0006768096986799738
train loss:0.002813804979024719
train loss:0.00016610356994522358
train loss:0.0011193057069199342
train loss:0.01075952829682539
train loss:0.00041427063602121866
train loss:0.0023375241333908926
train loss:0.0057465878621312805
train loss:0.0007455313231503203
train loss:0.0011204404572436712
train loss:0.004013670395607166
train loss:0.00325573512

train loss:0.0031042109546828424
train loss:0.001839559682464494
train loss:0.00377455971472909
train loss:0.0007738527958631468
train loss:0.0027144934022103016
train loss:0.00042154476743520153
train loss:0.0012697658473393247
train loss:0.00012087094723145025
train loss:0.006759060603508028
train loss:0.0002357201928788123
train loss:0.0035903656339431006
train loss:0.0034698902577538393
train loss:8.413171100385703e-05
train loss:0.0007771255017428413
train loss:0.0004632525182839753
train loss:0.0006278085609600022
train loss:0.00010375400796972964
train loss:0.0009027050647269389
train loss:0.002059073217508594
train loss:0.0025834319462926707
train loss:0.0002758249809288846
train loss:0.0034627301080841118
train loss:0.0005962753315122868
train loss:0.0005586427586519116
train loss:0.0010616205226633816
train loss:0.001544455352612298
train loss:0.00033999225750859003
train loss:0.0003779762915216132
train loss:0.008716505946623695
train loss:0.0009555672673276449
train loss:0.

train loss:0.00023057273209895114
train loss:0.0003391692562192343
train loss:0.00013802203372192506
train loss:5.99330623538464e-05
train loss:0.0032714162554423744
train loss:0.0020983147315032564
train loss:0.00046739768293305455
train loss:0.006952831414334416
train loss:0.0011942080151932913
train loss:0.00257632478840259
train loss:0.00024071778450100105
train loss:0.00019500320039122202
train loss:0.0038728984952878624
train loss:8.943023357724997e-05
train loss:0.0001394394336638456
train loss:0.0004226705129658847
train loss:0.002287518028134751
train loss:0.0003476938100540957
train loss:0.0011555545574344005
train loss:0.00022596435121533055
train loss:0.002233443438533995
train loss:0.0015848999335590342
train loss:0.0007104037360139524
train loss:0.0007107636695894026
train loss:0.0016571181877140665
train loss:2.9977077080511344e-05
train loss:0.01005775886384961
train loss:0.0006830948286687345
train loss:0.006548280314067734
train loss:0.0007130824100534017
train loss:0

train loss:0.0011315154957450275
train loss:0.0004157895829736195
train loss:0.008887780877853347
train loss:0.002645449052950352
train loss:0.0013905562564327394
train loss:0.00034265039472649217
train loss:0.0012490963202355608
train loss:0.0005320176690109692
train loss:0.011212596861790918
train loss:0.0009948362685767707
train loss:0.0011356603474510073
train loss:0.0024010986263499744
train loss:0.003628024078247015
train loss:0.00017246335805674875
train loss:0.005956356683302058
train loss:0.00042072411456587424
train loss:0.001306227623057215
train loss:0.001112591352519981
train loss:0.0008719719533978942
train loss:0.00029367860073209565
train loss:0.0005471116378031618
train loss:0.0008105348101270807
train loss:0.0005426922271497713
train loss:0.0030716307741720235
train loss:0.004272892524991076
train loss:0.0016039845036032213
train loss:0.0011214762416439329
train loss:0.003637172195731617
train loss:0.0017275806978120053
train loss:0.0023968310446725975
train loss:0.00

train loss:0.0009023212482822258
train loss:0.0025557558444234013
train loss:0.0022824743973095594
train loss:3.298647113258616e-05
train loss:0.0009899511438720351
train loss:0.00042676534475874587
train loss:0.0003204160684600838
train loss:0.0018508045618867528
train loss:0.0004697239391907989
train loss:0.00076345352692752
train loss:0.001603517842146148
train loss:2.9507210142246795e-05
train loss:0.0003990101742167341
train loss:0.0036227073378946534
train loss:0.001381243920054607
train loss:0.0015783002584339426
train loss:0.00036081227147556253
train loss:0.0015975026068642476
train loss:5.808076465825798e-05
train loss:0.0011677661218973197
train loss:3.668027124856461e-05
train loss:0.00016520464297381412
train loss:0.0005989091393511759
train loss:0.00030484301431115557
train loss:0.00012095966138059362
train loss:0.0005459002058858694
train loss:3.719541665590578e-05
train loss:0.0002629130758482612
train loss:0.00028462037011144016
train loss:0.00044625859234865835
train 